In [1]:
import pandas as pd #data manipulation
import numpy as np

In [2]:
from sklearn.feature_extraction import DictVectorizer #modelling
from sklearn.linear_model import LinearRegression #modelling
from sklearn.metrics import mean_squared_error #modelling

In [3]:
# Jupyter display env setup
pd.set_option('display.max_rows', 200)

pd.options.display.max_columns

pd.options.display.max_colwidth

pd.options.display.precision = 3

pd.options.plotting.backend = "plotly"

np.random.seed(42)

In [4]:
# load the data for January 2022
df_train = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')

# and quickly inspect the size of the table
print(df_train.shape)

# Question 1: Read the data for January. How many columns are there?
print(df_train.shape[1])

df_train.head()

(2463931, 19)
19


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [8]:
# create the response variable: duration (in minutes)
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime

df_train.duration = df_train.duration.dt.total_seconds()/60

In [13]:
# Question 2: STD duration in Jan 2022

df_train.duration.std().round(2)

46.45

In [14]:
# Question 3: Fraction of the records left after dropping the outliers
df_train_noOutliers = df_train.query("duration >= 1 & duration <= 60")

df_train_noOutliers.shape[0]/df_train.shape[0] * 100

98.27547930522405

In [16]:
df_train_noOutliers.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [25]:
# subset just the locationID columns
PU_DO = df_train_noOutliers[['PULocationID', 'DOLocationID']]

PU_DO.head()

,PULocationID,DOLocationID
0,142,236
1,236,42
2,166,166
3,114,68
4,68,163


In [26]:
# set appropriate data type
PU_DO = PU_DO.astype(str)

PU_DO.dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [27]:
# turn into a list of dicts
PU_DO_dicts = PU_DO.to_dict(orient='records')
PU_DO_dicts

[{'PULocationID': '142', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '42'},
 {'PULocationID': '166', 'DOLocationID': '166'},
 {'PULocationID': '114', 'DOLocationID': '68'},
 {'PULocationID': '68', 'DOLocationID': '163'},
 {'PULocationID': '138', 'DOLocationID': '161'},
 {'PULocationID': '233', 'DOLocationID': '87'},
 {'PULocationID': '238', 'DOLocationID': '152'},
 {'PULocationID': '166', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '141'},
 {'PULocationID': '141', 'DOLocationID': '229'},
 {'PULocationID': '114', 'DOLocationID': '90'},
 {'PULocationID': '234', 'DOLocationID': '113'},
 {'PULocationID': '246', 'DOLocationID': '79'},
 {'PULocationID': '43', 'DOLocationID': '140'},
 {'PULocationID': '239', 'DOLocationID': '151'},
 {'PULocationID': '148', 'DOLocationID': '141'},
 {'PULocationID': '237', 'DOLocationID': '107'},
 {'PULocationID': '7', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '263'},
 {'PULocationID': '263', 'DOLoc

In [28]:
#Question 4: Dimensionality after OHE
dv = DictVectorizer()
PU_DO_ohe = dv.fit_transform(PU_DO_dicts)

PU_DO_ohe.shape

(2421440, 515)

In [29]:
# Build a vanilla LR
Y_train = df_train_noOutliers.duration.values

X_train = PU_DO_ohe

lr = LinearRegression()
lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_train)

In [30]:
actuals_preds = pd.DataFrame({'Actual': Y_train, 'Predicted': Y_pred})

actuals_preds.head()

,Actual,Predicted
0,17.817,9.455
1,8.400,16.135
2,8.967,13.817
3,10.033,10.436
4,37.533,11.086


In [35]:
#Question 5: RMSE on train
print(mean_squared_error(actuals_preds.Actual, actuals_preds.Predicted, squared=False).round(2))

6.99


In [36]:
# load Feb 2022 data and create response var: duration
df_val = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime

df_val.duration = df_val.duration.dt.total_seconds()/60


In [38]:
# according to instructions: follow similar preprocessing as in the train subset
df_val = df_val.query("duration >= 1 & duration <= 60")

PU_DO_val = df_val[['PULocationID', 'DOLocationID']]

PU_DO_val = PU_DO_val.astype(str)

print(PU_DO_val.dtypes)

PU_DO_val = PU_DO_val.to_dict(orient='records')

PU_DO_val = dv.transform(PU_DO_val)

print(PU_DO_val.shape[1])

PULocationID    object
DOLocationID    object
dtype: object
515


In [39]:
# Produce predictions on the val df
Y_val = df_val.duration.values

Y_pred_val = lr.predict(PU_DO_val)

In [40]:
actual_preds_val = pd.DataFrame({'Actual_val': Y_val, 'Predicted_val': Y_pred_val})


#Question 6: RMSE on validation
print(mean_squared_error(actual_preds_val.Actual_val, actual_preds_val.Predicted_val, 
                         squared=False).round(2))

7.79
